In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from pathlib import Path
import sys
import glob
from os.path import join
import geopandas as gpd
import pandas as pd
import numpy as np

/tmp/ipykernel_3656689/863041077.py:6: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


In [3]:
# It could make sense to have a lib/ style directory
# like PLACES has for common functionality
# and this code block would be useful there for getting
# a fr() path

# Get the absolute path to the precal_hazard directory
# Which is two directories above notebooks/exploration/
abs_dir = os.path.abspath(Path(os.getcwd()).parents[1])
# Get raw data directory
fr = join(abs_dir, 'data', 'raw')
# Get interim data directory
fi = join(abs_dir, 'data', 'interim')
# Get processed data directory
fp = join(abs_dir, 'data', 'processed')

# Load Data

In [41]:
# Load structures (linked to hazard)
# Probably more reasonable to load hazard as well
# Especially as we go on to take in more hazard grids
EXP_DIR = join(fp, 'exposure')
EXP_FILEP = join(EXP_DIR, 'nsi_res.gpkg')
nsi_res = gpd.read_file(EXP_FILEP)

# Load depths
DEPTH_FILEP = join(EXP_DIR, 'depths.pqt')
depths = pd.read_parquet(DEPTH_FILEP)

# Load damage functions
# Filepath to NACCS depth damage functions
VUL_DIR = join(fp, 'vulnerability')
# Read ddfs
naccs = pd.read_csv(join(VUL_DIR, 'naccs_ddfs.csv'))

# Subset to 1/2 story residences

In [42]:
# Add #story and wb or nb to RES3 homes
# Store NB or WB indexed to RES3 homes based on B,C and N found_type
# Get num_story + 'S' 
# Merge these and then add to occtype for RES3 homes

# Start with index of res3 homes
res3_ind = nsi_res['occtype'].str[:4] == 'RES3'
# Get subsetted df
res3 = nsi_res.loc[res3_ind]

# For this subset
# If found_type == B, then WB
# Else then NB
res3b = np.where(res3['found_type'] == 'B',
                 'WB',
                 'NB')
# For this subset
# Get num_story + 'S'
res3s = res3['num_story'].astype(str) + 'S'

# Adjust occtype column for these homes in nsi_res
nsi_res.loc[res3_ind, 'occtype'] = res3['occtype'] + '-' + res3s + res3b

# For this case-study, don't use multifamily residences
# Drop any RES3 buildings
nsi_res_f = nsi_res.loc[~res3_ind]

# For this case-study, don't use any building with more 
# than 2 stories
res1_3s_ind = nsi_res_f['num_story'] > 2
# Final exposure data
res_f = nsi_res_f.loc[~res1_3s_ind]



# Calculate losses

## Get inundation depth relative to FFE

In [43]:
# Subset for dataframe of columns needed for loss estimates and merging
res_loss_cols = ['fd_id', 'occtype', 'found_ht', 'found_type', 'val_struct']
res_loss_temp = res_f.loc[:,res_loss_cols].set_index('fd_id')

# Merge res_loss with depths
res_loss = res_loss_temp.merge(depths, on='fd_id', how='inner')

In [44]:
# Columns with depths
depth_cols = [x for x in res_loss.columns if 'depth' in x]

# Subtract foundation height from depth relative to grade
res_loss.loc[:, depth_cols] = res_loss.loc[:, depth_cols].sub(res_loss['found_ht'], axis=0)

# Round depth to nearest 10th of a foot
res_loss.loc[:, depth_cols] = res_loss.loc[:, depth_cols].round(1)

# For basement homes, we need to adjust to the depth of the basement
# In the NACCS region, basements are assumed to have
# -8 ft relative to FFE as where damage can be incurred
# So we'll adjust the depth by 8
bhomes = res_loss['found_type'] == 'B'
res_loss.loc[bhomes, depth_cols] = res_loss.loc[bhomes, depth_cols].sub(8, axis=0)

## Get the reldam from triangular distribution linked to structure

In [45]:
# For each occtype and damcat combo we need to 
# linearly interpolate depths at 10th of foot increments
# and get the reldam

# Store this in a dataframe with the following shape
# occtype & depth_ft index, columns reldam_min/ml/max
# This way we can merge on occtype and depth_ft and 
# Add the correct reldam_min/ml/max as columns to the structures dataframe
# You then can call the triangular random generator on those columns
# using apply and automatically get the reldam associated with 
# the structure

# Get RES1 DDFs
naccs_r = naccs.loc[naccs['occtype'].str[:4] == 'RES1']

# Loop through occtype, damcat pairs (groupby)
# Subset the naccs_r ddfs
# Get a full index
# Interpolate reldam between the 10th of foot increments
# Create a new fully interpolated dataframe
# Append to a list of ddfs
# After loop, concat these ddfs

# List for interpolated ddfs
ddf_list = []

# Loop through occtype, damcat pairs
for n, gb in naccs_r.groupby(['occtype', 'damcat']):
    # Get occtype and damcat
    occtype = n[0]
    damcat = n[1]
    
    # Get ddf
    ddf = naccs_r.loc[(naccs_r['occtype'] == occtype) &
                      (naccs_r['damcat'] == damcat)]
    
    # Get range
    min_d = ddf['depth_ft'].min()
    max_d = ddf['depth_ft'].max()
    
    # Get full index
    # Round to nearest 10th of foot
    # Add extra 10th of foot for inclusive range
    full_ind = pd.Index(np.arange(min_d, max_d + .1, .1).round(1))
    
    # Subset ddf to depth_ft and reldam
    # Set index to depth_ft, reindex on full_ind
    # Interpolate, reset & rename index, then add back occtype and damcat
    keepcols = ['depth_ft', 'reldam']
    ddf = ddf[keepcols].set_index('depth_ft')
    ddf_new = ddf.reindex(full_ind).interpolate().reset_index()
    # Update DDF depth_ft name to match naming convention in structure data
    updatecols = ['depth_ffe', 'reldam']
    ddf_new.columns = updatecols
    ddf_new['occtype'] = occtype
    ddf_new['damcat'] = damcat
    ddf_list.append(ddf_new)
# Concat for dataframe of ddfs
naccs_r_f = pd.concat(ddf_list, axis=0)
# pivot the dataframe so that you get reldamMin, 
# reldamML and reldamMax cols
naccs_r_f = naccs_r_f.pivot(index=['depth_ffe', 'occtype'],
                            columns=['damcat'],
                            values='reldam').reset_index()


In [53]:
# Melt dataframe so that we can easily apply damage curves
# Retain occtype, val, depths, and reset index
res_loss_f = res_loss.drop(columns=['found_ht', 'found_type']).reset_index()
res_loss_f = res_loss_f.melt(id_vars=['fd_id', 'occtype', 'val_struct'],
                             var_name='param',
                             value_name='depth')
# Update parameterization to just str after depth_
res_loss_f['param'] = res_loss_f['param'].str.split('_').str[1]

In [54]:
res_loss_f.head()

,fd_id,occtype,val_struct,param,depth
0,570201393,RES1-1SWB,438922.821,0.0375,-10.0
1,570201400,RES1-2SWB,558333.600,0.0375,-10.0
2,570201401,RES1-2SWB,411230.358,0.0375,-10.0
3,570201402,RES1-2SWB,340946.179,0.0375,-10.0
4,570201403,RES1-2SWB,447718.209,0.0375,-10.0


In [55]:
# Merge DDFs into the structure data
res_loss_f = res_loss_f.merge(naccs_r_f,
                              left_on=['depth', 'occtype'],
                              right_on=['depth_ffe', 'occtype'],
                              how='left')
# Drop depth_ffe
# Fill na with 0
res_loss_f = res_loss_f.drop(columns=['depth_ffe'])
res_loss_f.loc[:,['ML', 'Max', 'Min']] = res_loss_f.loc[:,['ML', 'Max', 'Min']].fillna(0)


# Helper function for drawing triangular distribution from
# Min, ML, Max
def tri_rd(mindam, mldam, maxdam):
    # Function will throw value error if left & right are equal
    # In this case, there is no distribution to draw from anyway
    # So you can just return any of the values
    if mindam == maxdam:
        return mindam
    return np.random.default_rng().triangular(left=mindam,
                                              mode=mldam,
                                              right=maxdam)

# Get reldam from triangular distribution from (Min, ML, Max)
res_loss_f['reldam'] = res_loss_f.apply(lambda x: tri_rd(x['Min'],
                                                         x['ML'],
                                                         x['Max']),
                                        axis=1)

# Get structure damage
res_loss_f['structdam'] = res_loss_f['reldam'] * res_loss_f['val_struct']

In [ ]:
# Group by parameterizations
# Sum over damages to structures
# Look at distribution of overall damages
res_loss_f.groupby(['param'])['structdam'].sum().describe().astype(int)

In [ ]:
# As of now, not writing out any of the damages to file
# The damages are calculated in memory